# Avance 1 – PI 5  
## Limpieza, Transformación y Modelo Relacional

## 1. Principales insights y problemas encontrados

### Calidad general de los datos  
Durante la exploración inicial de los archivos se observó que los datos no presentaban valores nulos relevantes ni inconsistencias en campos críticos como identificadores, montos, cantidades o estados.  
Los valores faltantes identificados correspondían exclusivamente a campos opcionales (descripciones, comentarios, fechas secundarias y datos de direcciones), lo cual se considera aceptable para este tipo de estructura.
Se descubrió que no coincide el total de las órdenes con la suma de totales de los detalles. Por lo que se realizará una actualización para que coincidan.



### Estructura del dataset  
Los datos representan un modelo típico de comercio electrónico con tablas que corresponden a:
- Usuarios, categorías y productos.  
- Órdenes y detalle de órdenes.  
- Métodos de pago y pagos realizados.  
- Carrito de compras.  
- Reseñas de productos.  
- Direcciones de envío.

### Problemas detectados  
Los datasets originales no incluían claves primarias definidas de forma estándar en todas las tablas, lo cual dificulta el diseño relacional y la carga en un motor de base de datos.

Decisión tomada: agregar una columna `id` autonumérica como clave primaria en cada tabla para estandarizar el esquema y facilitar el establecimiento de relaciones.

Otro problema detectado es que el esquema permite que un usuario tengo varias direcciones de envío, pero este dato no está en la tabla de órdenes, por lo tanto se declara como una relación 1 a 1.

---

## 2. Decisiones tomadas respecto a limpieza y transformación

### Creación de claves primarias autonuméricas  
En todas las tablas se incorporó una columna:

```sql
id SERIAL PRIMARY KEY
```

Esta columna actúa como identificador único y permite establecer las relaciones necesarias en un esquema relacional.

### Estandarización de tipos de datos  
Para garantizar consistencia y compatibilidad con PostgreSQL, se definieron los siguientes tipos:
- Identificadores → `INTEGER`.  
- Precios y montos → `NUMERIC(12,2)` o `NUMERIC(14,2)` según la tabla.  
- Cantidades → `INTEGER`.  
- Fechas → `TIMESTAMP`.  
- Campos de texto → `VARCHAR` o `TEXT`.

### Manejo de valores nulos  
- Se reemplazaron valores `NaN` por `NULL` en PostgreSQL.  
- Las fechas opcionales se procesaron mediante `errors="coerce"` para evitar fallas en el parseo.  
- Los campos de texto opcionales se almacenaron como `NULL` cuando correspondía.

### Normalización de textos  
- Los correos electrónicos fueron estandarizados mediante `.strip()` para eliminar espacios.  
- Otros campos textuales se normalizaron para garantizar coherencia entre tipos de datos.

### Estructura de carga a la base de datos  
- Se utilizó un esquema denominado `staging` para organizar las tablas de forma clara dentro del motor SQL.  
- El proceso de carga respetó un orden lógico: primero tablas maestras (usuarios, categorías, métodos de pago), luego productos y órdenes, y finalmente las tablas dependientes (detalle de órdenes, reseñas, historial de pagos, direcciones, carrito).

---

## 3. Tablas finales resultantes

A continuación se detallan las tablas, sus columnas principales, tipos de datos y relaciones preliminares, según el diseño relacional y el diagrama ER correspondiente.

DIAGRAMA ER
![alt text](image-1.png)

### 3.1. Tabla `usuarios`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| nombre | VARCHAR(100) |
| apellido | VARCHAR(100) |
| dni | VARCHAR(20) |
| email | VARCHAR(200) |
| contrasena | VARCHAR(200) |

Relaciones:  
- `usuarios` 1–N `ordenes`  
- `usuarios` 1–N `direcciones_envio`  
- `usuarios` 1–N `carrito`  
- `usuarios` 1–N `resenas_productos`

---

### 3.2. Tabla `categorias`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| nombre | VARCHAR(100) |
| descripcion | TEXT |

Relación:  
- `categorias` 1–N `productos`

---

### 3.3. Tabla `productos`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| nombre | VARCHAR(200) |
| descripcion | TEXT |
| precio | NUMERIC(12,2) |
| stock | INTEGER |
| categoria_id | INTEGER |

Relaciones:  
- `productos` 1–N `detalle_ordenes`  
- `productos` 1–N `carrito`  
- `productos` 1–N `resenas_productos`

---

### 3.4. Tabla `ordenes`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| usuario_id | INTEGER |
| fecha_orden | TIMESTAMP |
| total | NUMERIC(14,2) |
| estado | VARCHAR(50) |

Relaciones:  
- `ordenes` 1–N `detalle_ordenes`  
- `ordenes` 1–N `ordenes_metodos_pago`  
- `ordenes` 1–N `historial_pagos`

---

### 3.5. Tabla `detalle_ordenes`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| orden_id | INTEGER |
| producto_id | INTEGER |
| cantidad | INTEGER |
| precio_unitario | NUMERIC(12,2) |

---

### 3.6. Tabla `direcciones_envio`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| usuario_id | INTEGER |
| calle | VARCHAR(200) |
| ciudad | VARCHAR(120) |
| departamento | VARCHAR(120) |
| provincia | VARCHAR(120) |
| distrito | VARCHAR(120) |
| estado | VARCHAR(120) |
| codigo_postal | VARCHAR(20) |
| pais | VARCHAR(80) |

---

### 3.7. Tabla `carrito`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| usuario_id | INTEGER |
| producto_id | INTEGER |
| cantidad | INTEGER |
| fecha_agregado | TIMESTAMP |

---

### 3.8. Tabla `metodos_pago`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| nombre | VARCHAR(120) |
| descripcion | TEXT |

---

### 3.9. Tabla `ordenes_metodos_pago`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| orden_id | INTEGER |
| metodo_pago_id | INTEGER |
| monto_pagado | NUMERIC(14,2) |

---

### 3.10. Tabla `resenas_productos`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| usuario_id | INTEGER |
| producto_id | INTEGER |
| calificacion | INTEGER |
| comentario | TEXT |
| fecha | TIMESTAMP |

---

### 3.11. Tabla `historial_pagos`

| Columna | Tipo |
|--------|------|
| id | SERIAL PRIMARY KEY |
| orden_id | INTEGER |
| metodo_pago_id | INTEGER (NULLABLE) |
| monto | NUMERIC(14,2) |
| fecha_pago | TIMESTAMP |
| estado_pago | VARCHAR(50) |

---

## 4. Resumen de relaciones preliminares

- `usuarios` 1–N `ordenes`  
- `usuarios` 1–1 `direcciones_envio`  
- `usuarios` 1–N `carrito`  
- `usuarios` 1–N `resenas_productos`  
- `categorias` 1–N `productos`  
- `productos` 1–N `detalle_ordenes`  
- `productos` 1–N `carrito`  
- `productos` 1–N `resenas_productos`  
- `ordenes` 1–N `detalle_ordenes`  
- `ordenes` 1–N `ordenes_metodos_pago`  
- `ordenes` 1–N `historial_pagos`  
- `metodos_pago` 1–N `ordenes_metodos_pago`  
- `metodos_pago` 1–N `historial_pagos`
